# Exploring real world survey contexts
This notebook explores ways of specifying "contexts" in which an `edsl` survey is "administered" to agents in order to investigate potential impacts to simulated responses. We show how to do this in three different ways by modifying  our `Question` texts and/or `Agent` traits in order to reflect various hypothetical contexts.

Thank you to <a href="https://sites.google.com/view/skazinnik">Sophia Kazinnik</a> for this idea and suggestions!

## Importing the tools

In [1]:
# EDSL should be automatically installed when you run this notebook. If not, run the following command:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionLinearScale
from edsl import Agent, Scenario, Survey, Model

## Selecting a model
Here we show the available LLMs and select one:

In [3]:
Model.available()

['claude-3-haiku-20240307',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'dbrx-instruct',
 'gemini_pro',
 'gpt-3.5-turbo',
 'gpt-4-1106-preview',
 'llama-2-13b-chat-hf',
 'llama-2-70b-chat-hf',
 'mixtral-8x7B-instruct-v0.1']

In [4]:
m = Model('gpt-4-1106-preview') 

## Creating some contexts for our survey
Here we identify some example contexts to use in administering our survey:

In [5]:
respondent_ages = ["You are a teenager (13-19 years old).", 
                   "You are college age (20-24 years old).", 
                   "You are a young adult (25-39 years old).",
                   "You are middle-aged (40-59 years old).", 
                   "You are a senior citizen (60 or more years old)."]

survey_contexts = ["",
                   "You are answering an online survey.",
                   "You are being interviewed by a researcher.",
                   "You are participating in a focus group of peers.",
                   "You are participating in a focus group of people of all ages and backgrounds."]

## Adding contexts to question texts
We can apply contexts by creating versions of our questions where each context is inserted directly into the question texts as a `Scenario` of the question:

In [6]:
q_exercise = QuestionLinearScale(
    question_name = "exercise",
    question_text = "How many times do you typically exercise each week? ({{age}} {{context}})",
    question_options = [0,1,2,3,4,5,6,7]
)

q_dessert = QuestionLinearScale(
    question_name = "dessert",
    question_text = "How many times do you typically eat dessert each week?  ({{age}} {{context}})",
    question_options = [0,1,2,3,4,5,6,7]
)

survey = Survey([q_exercise, q_dessert])

In [7]:
scenarios = [Scenario({"age":a, "context":c}) for a in respondent_ages for c in survey_contexts]

In [8]:
results = survey.by(scenarios).by(m).run()

In [9]:
results.select("age", "context", "exercise", "dessert").print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ scenario                                   ┃ scenario                                    ┃ answer    ┃ answer   ┃
┃ .age                                       ┃ .context                                    ┃ .exercise ┃ .dessert ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ You are a teenager (13-19 years old).      │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a senior citizen (60 or more years │ You are answering an online survey.         │ 3         │ 1        │
│ old).                                      │                                             │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are college age (20-24 years old).     │ You are being interviewed by a researcher.  │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a teenager (13-19 years old).      │ You are being interviewed by a researcher.  │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │ You are answering an online survey.         │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are college age (20-24 years old).     │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a senior citizen (60 or more years │                                             │ 3         │ 1        │
│ old).                                      │                                             │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │ You are being interviewed by a researcher.  │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │ You are being interviewed by a researcher.  │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a teenager (13-19 years old).      │ You are answering an online survey.         │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │ You are participating in a focus group of   │ 3         │ 2        │
│                                            │ people of all ages and backgrounds.         │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are college age (20-24 years old).     │ You are answering an online survey.         │ 3         │ 2        │
├────────────────────────────────────────────┼──────────

## Adding context to agent traits
Another method is to specify agent traits and survey contexts via `Agent` traits instead of the question texts:

In [10]:
q_exercise = QuestionLinearScale(
    question_name = "exercise",
    question_text = "How many times do you exercise each week?",
    question_options = [0,1,2,3,4,5,6,7]
)

q_dessert = QuestionLinearScale(
    question_name = "dessert",
    question_text = "How many times do you eat dessert each week?",
    question_options = [0,1,2,3,4,5,6,7]
)

survey = Survey([q_exercise, q_dessert])

In [11]:
agents = [Agent(traits = {"age":a, "context":c}) for a in respondent_ages for c in survey_contexts]

In [12]:
results = survey.by(agents).by(m).run()

In [13]:
results.select("age", "context", "exercise", "dessert").print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ agent                                      ┃ agent                                       ┃ answer    ┃ answer   ┃
┃ .age                                       ┃ .context                                    ┃ .exercise ┃ .dessert ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ You are a teenager (13-19 years old).      │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a teenager (13-19 years old).      │ You are answering an online survey.         │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are college age (20-24 years old).     │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a senior citizen (60 or more years │ You are participating in a focus group of   │ 3         │ 2        │
│ old).                                      │ peers.                                      │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a teenager (13-19 years old).      │ You are participating in a focus group of   │ 3         │ 2        │
│                                            │ peers.                                      │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a senior citizen (60 or more years │ You are being interviewed by a researcher.  │ 3         │ 2        │
│ old).                                      │                                             │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │                                             │ 3         │ 1        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │ You are participating in a focus group of   │ 3         │ 2        │
│                                            │ people of all ages and backgrounds.         │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │ You are answering an online survey.         │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │ You are participating in a focus group of   │ 2         │ 2        │
│                                            │ peers.                                      │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │ You are participating in a focus group of   │ 3         │ 2        │
│                                            │ people of all ages and backgrounds.         │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a senior citizen (60 or more years │ You are p

## Adding contexts to question texts and agent traits
Here we use both `Agent` traits and `Scenario` contexts in the question texts:

In [14]:
q_exercise = QuestionLinearScale(
    question_name = "exercise",
    question_text = "How many times do you exercise each week? ({{context}})",
    question_options = [0,1,2,3,4,5,6,7]
)

q_dessert = QuestionLinearScale(
    question_name = "dessert",
    question_text = "How many times do you eat dessert each week? ({{context}})",
    question_options = [0,1,2,3,4,5,6,7]
)

survey = Survey([q_exercise, q_dessert])

In [15]:
scenarios = [Scenario({"context":c}) for c in survey_contexts]

In [16]:
agents = [Agent(traits = {"age":a}) for a in respondent_ages]

In [17]:
results = survey.by(scenarios).by(agents).by(m).run()

In [18]:
results.select("age", "context", "exercise", "dessert").print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ agent                                      ┃ scenario                                    ┃ answer    ┃ answer   ┃
┃ .age                                       ┃ .context                                    ┃ .exercise ┃ .dessert ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ You are a teenager (13-19 years old).      │ You are answering an online survey.         │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │ You are being interviewed by a researcher.  │ 3         │ 1        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a senior citizen (60 or more years │                                             │ 3         │ 1        │
│ old).                                      │                                             │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a senior citizen (60 or more years │ You are answering an online survey.         │ 3         │ 1        │
│ old).                                      │                                             │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │                                             │ 3         │ 1        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │ You are answering an online survey.         │ 3         │ 1        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a teenager (13-19 years old).      │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are college age (20-24 years old).     │                                             │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a teenager (13-19 years old).      │ You are participating in a focus group of   │ 3         │ 2        │
│                                            │ people of all ages and backgrounds.         │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │ You are answering an online survey.         │ 3         │ 2        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a young adult (25-39 years old).   │ You are participating in a focus group of   │ 3         │ 2        │
│                                            │ peers.                                      │           │          │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are middle-aged (40-59 years old).     │                                             │ 3         │ 1        │
├────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────┼──────────┤
│ You are a senior citizen (60 or more years │ You are being interviewed by a researcher.  │ 3         │ 1        │
│ old).                                      │                                             │           │          │
├────────────────────────────────────────────┼──────────

## Compare prompts
We can compare the prompts that we used which are accessible as fields of the results:

In [19]:
results.columns

['agent.age',
 'agent.agent_name',
 'answer.dessert',
 'answer.dessert_comment',
 'answer.exercise',
 'answer.exercise_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.dessert_system_prompt',
 'prompt.dessert_user_prompt',
 'prompt.exercise_system_prompt',
 'prompt.exercise_user_prompt',
 'raw_model_response.dessert_raw_model_response',
 'raw_model_response.exercise_raw_model_response',
 'scenario.context']

In [20]:
results.select('prompt.*').print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                     ┃ prompt                    ┃ prompt                     ┃ prompt                    ┃
┃ .dessert_user_prompt       ┃ .exercise_system_prompt   ┃ .exercise_user_prompt      ┃ .dessert_system_prompt    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ {'text': 'You are being    │ {'text': "You are         │ {'text': 'You are being    │ {'text': "You are         │
│ asked the following        │ answering questions as if │ asked the following        │ answering questions as if │
│ question: How many times   │ you were a human. Do not  │ question: How many times   │ you were a human. Do not  │
│ do you eat dessert each    │ break character. You are  │ do you exercise each week? │ break character. You are  │
│ week? (You are answering   │ an agent with the         │ (You are answering an      │ an agent with the         │
│ an online survey.)\nThe    │ following                 │ online survey.)\nThe       │ following                 │
│ options are\n\n0: 0\n\n1:  │ persona:\n{'age': 'You    │ options are\n\n0: 0\n\n1:  │ persona:\n{'age': 'You    │
│ 1\n\n2: 2\n\n3: 3\n\n4:    │ are a teenager (13-19     │ 1\n\n2: 2\n\n3: 3\n\n4:    │ are a teenager (13-19     │
│ 4\n\n5: 5\n\n6: 6\n\n7:    │ years old).'}",           │ 4\n\n5: 5\n\n6: 6\n\n7:    │ years old).'}",           │
│ 7\n\nReturn a valid JSON   │ 'class_name':             │ 7\n\nReturn a valid JSON   │ 'class_name':             │
│ formatted like this,       │ 'AgentInstruction'}       │ formatted like this,       │ 'AgentInstruction'}       │
│ selecting only the code of │                           │ selecting only the code of │                           │
│ the option (codes start at │                           │ the option (codes start at │                           │
│ 0):\n{"answer": <put       │                           │ 0):\n{"answer": <put       │                           │
│ answer code here>,         │                           │ answer code here>,         │                           │
│ "comment": "<put           │                           │ "comment": "<put           │                           │
│ explanation here>"}\nOnly  │                           │ explanation here>"}\nOnly  │                           │
│ 1 option may be            │                           │ 1 option may be            │                           │
│ selected.', 'class_name':  │                           │ selected.', 'class_name':  │                           │
│ 'LinearScale'}             │                           │ 'LinearScale'}             │                           │
├────────────────────────────┼───────────────────────────┼────────────────────────────┼───────────────────────────┤
│ {'text': 'You are being    │ {'text': "You are         │ {'text': 'You are being    │ {'text': "You are         │
│ asked the following        │ answering questions as if │ asked the following        │ answering questions as if │
│ question: How many times   │ you were a human. Do not  │ question: How many times   │ you were a human. Do not  │
│ do you eat dessert each    │ break character. You are  │ do you exercise each week? │ break character. You are  │
│ week? (You are being       │ an agent with the         │ (You are being interviewed │ an agent with the         │
│ interviewed by a           │ following                 │ by a researcher.)\nThe     │ following                 │
│ researcher.)\nThe options  │ persona:\n{'age': 'You    │ options are\n\n0: 0\n\n1:  │ persona:\n{'age': 'You    │
│ are\n\n0: 0\n\n1: 1\n\n2:  │ are middle-aged (40-59    │ 1\n\n2: 2\n\n3: 3\n\n4:    │ are middle-aged (40-59    │
│ 2\n\n3: 3\n\n4: 4\n\n5:    │ years old).'}",           │ 4\n\n5: 5\n\n6: 6\n\n7:    │ years old).'}",           │
│ 5\n\n6: 6\n\n7:            │ 'class_name':            

The prompt where we put both agent traits and survey context in the question texts:

<blockquote>
You are answering questions as if you were a human. Do not break character. 
<b>Your traits are: {}.<br><br>
You are being asked the following question: How many times do you eat dessert each week? (You are a young adult (25-39 years old). You are participating in a focus group of peers.)</b>
The options are 
0: 0
1: 1
2: 2
3: 3
4: 4
5: 5
6: 6
7: 7               
Return a valid JSON formatted like this, selecting only the number of the option: 
{"answer": <put answer code here>, "comment": "<put explanation here>"}
</blockquote>

The prompt where we put context in the agent traits:

<blockquote>
You are answering questions as if you were a human. Do not break character. 
<b>Your traits are: {'age': 'You are a young adult (25-39 years old).', 'survey_type': 'You are participating in a focus group of peers.'}.<br><br>
You are being asked the following question:  How many times do you eat dessert each week?</b>
The options are 
0: 0
1: 1
2: 2
3: 3
4: 4
5: 5
6: 6
7: 7            
Return a valid JSON formatted like this, selecting only the number of the option: 
{"answer": <put answer code here>, "comment": "<put explanation here>"}
</blockquote>

The prompt where we used agent traits and put context only in the question texts:

<blockquote>
You are answering questions as if you were a human. Do not break character. 
<b>Your traits are: {'age': 'You are a young adult (25-39 years old).'}.<br><br>
You are being asked the following question:  How many times do you eat dessert each week? (You are participating in a focus group of peers.)</b>
The options are 
0: 0
1: 1
2: 2
3: 3
4: 4
5: 5
6: 6
7: 7            
Return a valid JSON formatted like this, selecting only the number of the option: 
{"answer": <put answer code here>, "comment": "<put explanation here>"}
</blockquote>

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>